## 李沐 《动手学深度学习》学习笔记(三) 第二章 线性回归 第三节 线性回归简洁实现
实际上，由于数据迭代器、损失函数、优化器和神经网络层很常用，现代深度学习库都有实现这些组件
## 3.1 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 3.2 读取数据集
调用框架中现有的API来读取数据。将features和labels作为APi de 参数传递，并在实例化数据迭代器对象时指定batch_size。此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据

In [3]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [4]:
batch_size = 10;
data_iter = load_array((features, labels), batch_size)

为了验证是否正常工作，读取并打印第一个小批量样本。使用iter构造Python迭代器，并使用 next 从迭代器中获取第一项

In [5]:
next(iter(data_iter))

[tensor([[ 0.6190,  0.8266],
         [ 0.3382,  0.7921],
         [-0.2762,  0.0795],
         [ 0.3351,  0.4740],
         [-2.1380, -0.0902],
         [ 0.6070, -0.4707],
         [ 2.6408, -0.0664],
         [ 1.0313,  1.0903],
         [-0.4970, -0.8937],
         [ 0.2643,  0.7731]]),
 tensor([[2.6227],
         [2.1822],
         [3.4076],
         [3.2598],
         [0.2312],
         [7.0116],
         [9.7107],
         [2.5501],
         [6.2467],
         [2.0967]])]

## 3.3 定义模型
对于标准操作，可以使用框架的预定好的层，这使得我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。
* 首先定义一个模型变量 net(是一个 Sequential类的实例)，Sequential 类为串联在一起的多个层定义了一个容器。当给定输入数据，Sequential 实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，依此类推。
* 在Pytorch中，**全连接层**在 Linear类中定义。两个参数，第一个指定输入特征形状；第二个指定输出特征形状

In [6]:
# `nn` 是神经网络的缩写
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

## 3.4 初始化模型参数
在使用net之前，需要初始化模型参数。指定每个权重参数应该服从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为0
* 通过net[0]选择网络中的第一个图层
* 然后使用weight.data和bias.data方法访问参数
* 再使用替换方法normal_和fill_来重写参数值

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 3.5 定义损失函数
计算均方误差使用的是MSELoss类，也成为平方$L_2$范数。默认情况下，返回所有样本损失的平均值

In [8]:
loss = nn.MSELoss()

## 3. 6 定义优化算法
小批量随机梯度下降算法是一种优化神经网络的标准工具，PyTorch在optim模块中实现了该算法的许多变种。
当实例化SGD随机梯度下降法实例时，需要指定优化的参数（通过net.parameters()从模型中获得）以及优化算法所需的超参数字典

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 3.7 训练
在每一个迭代周期里，我们将完整遍历一次数据集（train_data），不停地从中获取一个小批量的输入和相应的标签。对于每一个小批量，会进行以下步骤：
* 通过调用net(X) 生成预测并计算损失l(正向传播)
* 通过进行反向传播来计算梯度
* 通过调用优化器来更新模型参数
为了更好的衡量训练效果，计算每个迭代周期后损失，并打印它来监控训练过程

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000257
epoch 2, loss 0.000102
epoch 3, loss 0.000099


比较生成数据集的真实参数和通过有限数据训练获得的模型参数。
* 首先从net访问所需的层
* 然后读取该层的权重和偏置

In [12]:
w = net[0].weight.data
print(f'w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print(f'b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0001, -0.0003])
b的估计误差： tensor([-0.0005])


## 3.8 小结
* 可以使用 PyTorch 的高级API更简洁地实现模型
* 在 Pytorch 中，data 模块提供了数据处理工具，nn 模块定义了大量的神经网络层和常见的损失函数
* 可以通过_结尾的方法将参数替换，从而初始化参数